In [1]:
#Q13.

import requests
import numpy as np


def download_data(url,filename):    #download file as data
    result = requests.get(url)
    result.raise_for_status()
    with open(filename,'wb') as FILE:
        for chunk in result.iter_content(102400):
            FILE.write(chunk)
            
url01 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_algo/hw4_train.dat'
url02 = 'https://www.csie.ntu.edu.tw/~htlin/mooc/datasets/mlfound_algo/hw4_test.dat'
download_data(url01, 'HW4_training_data.txt')
download_data(url02, 'HW4_test_data.txt')
    
    
def readout(filename):
    with open(filename,'r') as FILE:
        n, Dx, Dy = 0 ,[], []
        
        for chunk in FILE:
            X= chunk.split()   #split the line into a list of string
            X= [ float(X[j]) for j in range(len(X)) ]  #convert the string to numbers   
            X.insert(0,1)                #insert a constant as bias or as threshold
            Dx.append(X[0:len(X)-1])     #gather all data ## index len(X)-1 will not be included here
            Dy.append(X[len(X)-1])       #index len(X)-1 only
            n=n+1
        x = np.array(Dx)  #change list X into array x, and take transpose by .T
        y = np.array(Dy)
    print(x,n)            
    return x,y,n        

trainx, trainy, trainn = readout('HW4_training_data.txt')
testx, testy, testn    = readout('HW4_test_data.txt') 

[[1.         0.568304   0.568283  ]
 [1.         0.310968   0.310956  ]
 [1.         0.103376   0.103373  ]
 [1.         0.0531882  0.053218  ]
 [1.         0.97006    0.970064  ]
 [1.         0.0941873  0.0941707 ]
 [1.         0.655902   0.655892  ]
 [1.         0.370821   0.370839  ]
 [1.         0.558482   0.558476  ]
 [1.         0.849389   0.849383  ]
 [1.         0.796038   0.796051  ]
 [1.         0.723246   0.723252  ]
 [1.         0.571236   0.571254  ]
 [1.         0.385144   0.38512   ]
 [1.         0.877176   0.877168  ]
 [1.         0.74655    0.746552  ]
 [1.         0.0676164  0.0676087 ]
 [1.         0.0412524  0.0412649 ]
 [1.         0.851637   0.851661  ]
 [1.         0.586989   0.58698   ]
 [1.         0.661014   0.660994  ]
 [1.         0.587988   0.587968  ]
 [1.         0.257615   0.257628  ]
 [1.         0.680505   0.680485  ]
 [1.         0.895242   0.895257  ]
 [1.         0.381124   0.381139  ]
 [1.         0.314332   0.31433   ]
 [1.         0.157744   0.15

### $$\nabla E_{in}(W_{REG}) + \frac{2\lambda}{N}W_{REG} = 0   $$ $$W_{REG} \leftarrow (Z^TZ + \lambda I)^{-1}Z^T y$$

In [2]:
def error(w,datax,datay,datan):
    err = []
#   yhat = [np.dot(w,datax[i]) for i in range(datan)]  #
    yhat = datax.dot(w)

    for a,b in zip(yhat,datay):
        if a*b<=0:
            err.append(1)
        else:
            err.append(0)
    error = sum(err) /datan
    return error

In [3]:
#Q13.
lamb = 10

wreg = np.linalg.inv(np.dot(trainx.T,trainx) + lamb*np.identity(len(trainx[0]))).dot(trainx.T).dot(trainy)


Ein = error(wreg,trainx,trainy,trainn)
Eout = error(wreg,testx,testy,testn)
print(Ein,Eout)

0.05 0.045


In [4]:
#Q14.
Ein = 1
Eout = 1
for i in range(2,-11,-1):
    alpha = 10**i
    wreg = np.linalg.inv(np.dot(trainx.T,trainx) + alpha*np.identity(len(trainx[0]))).dot(trainx.T).dot(trainy)
    
    ein = error(wreg,trainx,trainy,trainn)
    eout = error(wreg,testx,testy,testn)
    
    if ein < Ein:  #leave the biggest lamb with the min Ein
        Ein = ein
        Eout = eout
        lamb = alpha
        
    print(lamb, Ein, Eout)
    

100 0.24 0.261
10 0.05 0.045
1 0.035 0.02
1 0.035 0.02
0.01 0.03 0.016
0.01 0.03 0.016
0.01 0.03 0.016
0.01 0.03 0.016
0.01 0.03 0.016
0.01 0.03 0.016
1e-08 0.015 0.02
1e-08 0.015 0.02
1e-08 0.015 0.02


In [5]:
#Q15.
Ein = 1
Eout = 1
for i in range(2,-11,-1):
    alpha = 10**i
    wreg = np.linalg.inv(np.dot(trainx.T,trainx) + alpha*np.identity(len(trainx[0]))).dot(trainx.T).dot(trainy)
    
    ein = error(wreg,trainx,trainy,trainn)
    eout = error(wreg,testx,testy,testn)
    
    if eout < Eout: #leave the biggest lamb with the min Eout
        Ein = ein
        Eout = eout
        lamb = alpha
        
    print(lamb, Ein, Eout)

100 0.24 0.261
10 0.05 0.045
1 0.035 0.02
0.1 0.035 0.016
0.1 0.035 0.016
0.1 0.035 0.016
0.1 0.035 0.016
0.1 0.035 0.016
0.1 0.035 0.016
1e-07 0.03 0.015
1e-07 0.03 0.015
1e-07 0.03 0.015
1e-07 0.03 0.015


In [6]:
#Q16.
k = 80                  #validation size
j = trainn-k            #train data size

Dtrainx, Dtrainy, Dtrainn = trainx[0:j], trainy[0:j], j 
Dvalx  , Dvaly  , Dvaln   = trainx[j:,], trainy[j:] , k
Etrain, Eval, Eout = 1, 1, 1


for i in range(2,-11,-1):
    alpha = 10**i
    wreg = np.linalg.inv(np.dot(Dtrainx.T,Dtrainx) + alpha*np.identity(len(Dtrainx[0]))).dot(Dtrainx.T).dot(Dtrainy)
    
    etrain = error(wreg,Dtrainx,Dtrainy,Dtrainn)
    evali = error(wreg,Dvalx,Dvaly,Dvaln)
    eout = error(wreg,testx,testy,testn)
    
    if etrain < Etrain: #leave the biggest lamb with the min Etrain
        Etrain = etrain
        Eval = evali
        Eout = eout
        lamb = alpha
        
    print(lamb, Etrain, Eval, Eout)

100 0.3416666666666667 0.4125 0.414
10 0.075 0.125 0.08
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1e-08 0.0 0.05 0.025
1e-08 0.0 0.05 0.025
1e-08 0.0 0.05 0.025


In [7]:
#Q17.
k = 80                  #validation size
j = trainn-k            #train data size

Dtrainx, Dtrainy, Dtrainn = trainx[0:j], trainy[0:j], j 
Dvalx  , Dvaly  , Dvaln   = trainx[j:,], trainy[j:] , k
Etrain, Eval, Eout = 1, 1, 1


for i in range(2,-11,-1):
    alpha = 10**i
    wreg = np.linalg.inv(np.dot(Dtrainx.T,Dtrainx) + alpha*np.identity(len(Dtrainx[0]))).dot(Dtrainx.T).dot(Dtrainy)
    
    etrain = error(wreg,Dtrainx,Dtrainy,Dtrainn)
    evali = error(wreg,Dvalx,Dvaly,Dvaln)
    eout = error(wreg,testx,testy,testn)
    
    if evali < Eval: #leave the biggest lamb with the min Etrain
        Etrain = etrain
        Eval = evali
        Eout = eout
        lamb = alpha
        
    print(lamb, Etrain,Eval, Eout)



100 0.3416666666666667 0.4125 0.414
10 0.075 0.125 0.08
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028
1 0.03333333333333333 0.0375 0.028


In [8]:
#Q18.

wreg = np.linalg.inv(np.dot(trainx.T,trainx) + lamb*np.identity(len(trainx[0]))).dot(trainx.T).dot(trainy)

Ein = error(wreg,trainx,trainy,trainn)
Eout = error(wreg,testx,testy,testn)
print(Ein,Eout)

0.035 0.02


V-fold cross-validation:random-partition of D to V equal parts,  
take V-1 for training and 1 for validation orderly.  
$$E_{CV}(H,A) = \frac{1}{V}\Sigma^V_{v=1}(g^-_V)$$

In [9]:
#Q19 & Q20.
V = 5
k = int(trainn/V)                  #size of each fold
E=0


Etrain, Eval, Eout, Ecv = 1, 1, 1, 1


for i in range(2,-11,-1):
    alpha = 10**i
    E=0
    for j in range(5):
        
        tempx ,tempy = trainx, trainy
        Dtrainx, Dtrainy, Dtrainn = np.delete(tempx,[range(j*k,(j+1)*k)],axis=0), np.delete(tempy,[range(j*k,(j+1)*k)],axis=0), (V-1)*k
        Dvalx  , Dvaly  , Dvaln   = trainx[j*k:(j+1)*k], trainy[j*k:(j+1)*k], k
        
        wreg = np.linalg.inv(np.dot(Dtrainx.T,Dtrainx) + alpha*np.identity(len(Dtrainx[0]))).dot(Dtrainx.T).dot(Dtrainy)

        #etrain = error(wreg,Dtrainx,Dtrainy,Dtrainn)
        evali = error(wreg,Dvalx,Dvaly,Dvaln)
        #eout = error(wreg,testx,testy,testn)
        E = E+evali
    ecv = E/V   #Q19.
    
    if ecv < Ecv: #leave the biggest lamb with the min Etrain   #Q20.
        lamb = alpha
        wreg = np.linalg.inv(np.dot(trainx.T,trainx) + alpha*np.identity(len(trainx[0]))).dot(trainx.T).dot(trainy)
        Etrain = error(wreg,trainx,trainy,trainn)
        Ecv = ecv
        Eout = error(wreg,testx,testy,testn)


    print(lamb, Etrain,Ecv, Eout)




100 0.24 0.29000000000000004 0.261
10 0.05 0.06 0.045
1 0.035 0.034999999999999996 0.02
1 0.035 0.034999999999999996 0.02
1 0.035 0.034999999999999996 0.02
1 0.035 0.034999999999999996 0.02
1 0.035 0.034999999999999996 0.02
1 0.035 0.034999999999999996 0.02
1 0.035 0.034999999999999996 0.02
1 0.035 0.034999999999999996 0.02
1e-08 0.015 0.03 0.02
1e-08 0.015 0.03 0.02
1e-08 0.015 0.03 0.02


In [10]:
A = np.array([[1,2,3],[4,5,6],[7,8,9]])
B = np.array([[4,5,6],[7,8,9],[10,11,12]])
np.append(A,np.array([[1,2,3]]),axis=0)

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9],
       [1, 2, 3]])

In [11]:
tempx ,tempy = trainx, trainy
j=3
V = 5
k = int(trainn/V)                  
Dtrainx, Dtrainy, Dtrainn = np.delete(tempx,[range(j*k,(j+1)*k)],axis=0), np.delete(tempy,[range(j*k,(j+1)*k)],axis=0) , (V-1)*k
Dvalx  , Dvaly  , Dvaln   = trainx[j*k:(j+1)*k], trainy[j*k:(j+1)*k], k

In [12]:
#Dtrainx